In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [49]:
def scrape_cards(start, stop):
    cards = []
    for pg in range(start, stop+1):
        url = f"https://scryfall.com/search?as=full&order=name&page={pg}&q=is%3Acommander&unique=cards"
        response = requests.get(url)

        if response.status_code == 200: # if we are successful
            print(f"Response sucessful. Scraping page {pg}")

            # scrape data - card image, name, mana value & images and whether it is double-sided.
            soup = BeautifulSoup(response.text, 'html.parser')

            # find card profiles, which contains all the card info
            card_profiles = soup.find_all('div', class_='card-profile')
            print("nr of card profiles", len(card_profiles))

            for card in card_profiles:
                # find image element
                image_element = card.find('img', class_='card')

                # find mana cost element
                
                total_mana_cost = 0 # initialise mana cost variable
                mana_cost_element = card.find('span', class_='card-text-mana-cost')
                if mana_cost_element:
                    

                    # Count the number of non-generic mana
                    for abbr in mana_cost_element.find_all('abbr', class_='card-symbol'):
                        mana_type = abbr['title']
                        if 'generic' in mana_type:
                            get_generic_val = abbr.get_text(strip=True, separator='').replace('{', '').replace('}', '')
                            if ValueError: # if the generic mana value is undefined (ie "whatever you want") or something weird, we don't include this card
                                continue
                            else:
                                total_mana_cost += int(get_generic_val)

                        else:
                            total_mana_cost += 1 # at the moment, we don't care what color it is, only the total cost

                # store relevant information
                card_data = {
                    'img_url' : image_element['src'],
                    'card_name' : image_element['title'],
                    'double-sided' : 'data-rotate' in image_element.attrs,
                    'cost' : total_mana_cost

                }

                # add to list of cards
                cards.append(card_data)

    return cards


In [54]:
cards = scrape_cards(78,110)

Response sucessful. Scraping page 78
nr of card profiles 20
Response sucessful. Scraping page 79
nr of card profiles 20
Response sucessful. Scraping page 80
nr of card profiles 20
Response sucessful. Scraping page 81
nr of card profiles 20
Response sucessful. Scraping page 82
nr of card profiles 20
Response sucessful. Scraping page 83
nr of card profiles 20
Response sucessful. Scraping page 84
nr of card profiles 20
Response sucessful. Scraping page 85
nr of card profiles 20
Response sucessful. Scraping page 86
nr of card profiles 20
Response sucessful. Scraping page 87
nr of card profiles 20
Response sucessful. Scraping page 88
nr of card profiles 20
Response sucessful. Scraping page 89
nr of card profiles 20
Response sucessful. Scraping page 90
nr of card profiles 20
Response sucessful. Scraping page 91
nr of card profiles 20
Response sucessful. Scraping page 92
nr of card profiles 20
Response sucessful. Scraping page 93
nr of card profiles 20
Response sucessful. Scraping page 94
nr 

In [55]:
len(cards)

660

In [56]:
df = pd.DataFrame(cards)

In [57]:
display(df)

,img_url,card_name,double-sided,cost
0,https://cards.scryfall.io/large/front/c/2/c23a...,"Ratchet, Field Medic // Ratchet, Rescue Racer ...",True,1
1,https://cards.scryfall.io/large/front/3/b/3b73...,"Raul, Trouble Shooter (Fallout #115)",False,2
2,https://cards.scryfall.io/large/front/f/2/f272...,"Ravos, Soultender (Commander 2016 #39)",False,2
3,https://cards.scryfall.io/large/front/6/9/6997...,"Rayami, First of the Fallen (Commander 2019 #48)",False,3
4,https://cards.scryfall.io/large/front/8/e/8ee6...,"Rayne, Academy Chancellor (Urza's Destiny #43)",False,1
...,...,...,...,...
655,https://cards.scryfall.io/large/front/e/a/ea1a...,"Zaffai, Thunder Conductor (Commander 2021 #4)",False,2
656,https://cards.scryfall.io/large/front/a/c/ac81...,"Zagras, Thief of Heartbeats (Zendikar Rising #...",False,2
657,https://cards.scryfall.io/large/front/8/8/880f...,"Zahid, Djinn of the Lamp (Commander Masters #136)",False,2
658,https://cards.scryfall.io/large/front/9/c/9c6d...,"Zalto, Fire Giant Duke (Adventures in the Forg...",False,2


In [58]:
df.to_csv('commander_cards_data_78_to_110.csv', index=False)